Second part Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import pandas as pd
import numpy as np
import geocoder 

# This function flatten a list
def flatten (list):
    res = []
    for sublist in list:
        for item in sublist:
            res.append(item)
    return res                       

I load the data into a pandas data frame called df and I clean the data

In [3]:
df = pd.read_csv("table-1.csv")

# I eliminate the "Not assigned" Borough 
df = df[df['Borough'] != 'Not assigned']
df.index = np.arange(len(df))

# I clean the "Not assigned" Neighbourhood column
ln = df[df.Neighbourhood == 'Not assigned'].index.values.tolist()

for index in ln:
    df.iloc[index]['Neighbourhood'] = df.iloc[index]['Borough']
    
# I get the list of all Postcode

lc = df.Postcode.unique().tolist()

todrop=[]

for jj in range(0,len(lc)):
    df2 = df[df.Postcode == lc[jj]]
    if len(df2) > 1:
        
        #I get the list of indices of df2
        nn = df2.index.tolist()
        
        #I extract the corresponding values in the list val
        val = []
        for ind in nn:
            val.append(df.iloc[ind]['Neighbourhood'])
        
        # I select the first no trivial index of nn and I put there the list
        num = nn.pop(0)
        df.iloc[num]['Neighbourhood']  = str(", ".join(val))
        
        #I keep the indices that I'd like to eliminate
        todrop.append(nn)
       
      
df.drop(flatten(todrop),inplace=True)
df.index = np.arange(len(df))
df.head(10)    

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


The geoconder package did not work very well so I decided to import the csv file with the coordinates. I create a new data frame called coo

In [9]:
coo = pd.read_csv('Geospatial_Coordinates.csv')
coo = coo.rename(columns={"Postal Code":"Postcode"})
coo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I merge the two dataframes on the column Postcode

In [10]:
tot = pd.merge(df,coo,on = 'Postcode')

In [23]:
#I print the head of the resulting data frame
tot.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [22]:
tot[tot.Postcode == 'M4M']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
54,M4M,East Toronto,Studio District,43.659526,-79.340923
